In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [2]:
train_datagen=ImageDataGenerator(
                               rotation_range=30,
                               width_shift_range=0.1,
                               height_shift_range=0.1,
                               shear_range=0.01,
                               zoom_range=[0.8, 1.25],
                               horizontal_flip=True,
                               vertical_flip=False,
                               fill_mode='reflect',
                               data_format='channels_last',
                               brightness_range=[0.5, 1.5]) #included in our dependencies

test_datagen = ImageDataGenerator(
                               rotation_range=30,
                               width_shift_range=0.1,
                               height_shift_range=0.1,
                               shear_range=0.01,
                               zoom_range=[0.8, 1.25],
                               horizontal_flip=True,
                               vertical_flip=False,
                               fill_mode='reflect',
                               data_format='channels_last',
                               brightness_range=[0.5, 1.5]) #included in our dependencies

In [3]:
image_size = (224,224)
batch_size = 10

training_data_dir = r'C:\Users\dvir1\Study\deep learning\project\train'
validation_data_dir = r'C:\Users\dvir1\Study\deep learning\project\validation'

train_generator=train_datagen.flow_from_directory(training_data_dir,
                                                 target_size=image_size,
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='categorical',
                                                 shuffle=True)

validation_generator = test_datagen.flow_from_directory(
                                        validation_data_dir,
                                         target_size=image_size,
                                         color_mode='rgb',
                                        class_mode = "categorical")

Found 7040 images belonging to 100 classes.
Found 1988 images belonging to 100 classes.


In [4]:
x_batch, y_batch = next(train_generator)


In [5]:
from tensorflow.keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential, Model 
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, Callback

In [6]:
base_model = Sequential()


In [7]:
base_model.add(Conv2D(128, (3,3), input_shape = (image_size[0], image_size[1], 3)))


In [8]:
base_model.add(Conv2D(128, (3,3)))

In [9]:
base_model.add(MaxPooling2D(pool_size=(2,2)))
base_model.add(Conv2D(256, (3,3)))
base_model.add(Conv2D(256, (3,3)))
model.add(MaxPooling2D(pool_size=(2,2)))

NameError: name 'model' is not defined

In [10]:
base_model.add(MaxPooling2D(pool_size=(2,2)))
base_model.add(Conv2D(256, (3,3)))
base_model.add(Conv2D(256, (3,3)))
base_model.add(MaxPooling2D(pool_size=(2,2)))


In [11]:
classCount = 100

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(classCount,activation='softmax')(x) #final layer with softmax activation

model_final=Model(inputs=base_model.input,outputs=preds)

In [12]:
model_final.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_input (InputLayer)    (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 222, 222, 128)     3584      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 220, 220, 128)     147584    
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 110, 110, 128)     0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 108, 108, 256)     295168    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 106, 106, 256)     590080    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 53, 53, 256)       0         
__________

In [13]:
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])


In [14]:
step_size_train=train_generator.n//train_generator.batch_size
print(step_size_train)

704


In [15]:
class LossAccHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.accuracy = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.accuracy.append(logs.get('acc'))
        
history = LossAccHistory()

In [16]:
model_final.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   validation_data = validation_generator,
                   epochs=10
                   ,callbacks = [history])

Epoch 1/10
704/704 [==============================] - 229s 325ms/step - loss: 5.0012 - acc: 0.0278 - val_loss: 4.5508 - val_acc: 0.0332
Epoch 2/10
704/704 [==============================] - 223s 317ms/step - loss: 4.4415 - acc: 0.0456 - val_loss: 4.3594 - val_acc: 0.0458
Epoch 3/10
704/704 [==============================] - 223s 317ms/step - loss: 4.3573 - acc: 0.0516 - val_loss: 4.2421 - val_acc: 0.0563
Epoch 4/10
704/704 [==============================] - 221s 314ms/step - loss: 4.2564 - acc: 0.0584 - val_loss: 4.2273 - val_acc: 0.0634
Epoch 5/10
704/704 [==============================] - 221s 314ms/step - loss: 4.2051 - acc: 0.0662 - val_loss: 4.2669 - val_acc: 0.0578
Epoch 6/10
704/704 [==============================] - 222s 315ms/step - loss: 4.1658 - acc: 0.0685 - val_loss: 4.2853 - val_acc: 0.0523
Epoch 7/10
704/704 [==============================] - 222s 315ms/step - loss: 4.1354 - acc: 0.0761 - val_loss: 4.0641 - val_acc: 0.0780
Epoch 8/10
704/704 [============================

In [17]:
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=2, verbose=1, mode='auto')

In [18]:
import time

networkfileName = "vgg16_{}.h5".format(int(time.time()))
checkpoint = ModelCheckpoint(networkfileName, monitor='val_acc', verbose=1,
                             save_best_only=True, save_weights_only=False, mode='auto', period=3)

In [19]:
history = LossAccHistory()
model_final.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   validation_data = validation_generator,
                   epochs=25
                   ,callbacks = [history, checkpoint, early])

Epoch 1/25
704/704 [==============================] - 223s 316ms/step - loss: 3.9682 - acc: 0.0942 - val_loss: 4.0684 - val_acc: 0.0795
Epoch 2/25
704/704 [==============================] - 222s 315ms/step - loss: 3.9101 - acc: 0.1001 - val_loss: 3.9686 - val_acc: 0.0971
Epoch 3/25
703/704 [============================>.] - ETA: 0s - loss: 3.8689 - acc: 0.1063
Epoch 00003: val_acc improved from -inf to 0.11167, saving model to vgg16_1549893243.h5
704/704 [==============================] - 223s 317ms/step - loss: 3.8684 - acc: 0.1061 - val_loss: 3.8757 - val_acc: 0.1117
Epoch 4/25
704/704 [==============================] - 223s 317ms/step - loss: 3.8231 - acc: 0.1178 - val_loss: 3.8392 - val_acc: 0.1172
Epoch 5/25
704/704 [==============================] - 223s 316ms/step - loss: 3.7808 - acc: 0.1203 - val_loss: 3.8552 - val_acc: 0.1112
Epoch 6/25
703/704 [============================>.] - ETA: 0s - loss: 3.7378 - acc: 0.1310
Epoch 00006: val_acc improved from 0.11167 to 0.11922, saving

In [20]:
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=1, mode='auto')

In [21]:
import time

networkfileName = "Gabis_{}.h5".format(int(time.time()))
checkpoint = ModelCheckpoint(networkfileName, monitor='val_acc', verbose=1,
                             save_best_only=True, save_weights_only=False, mode='auto', period=1)

In [22]:
history = LossAccHistory()
model_final.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   validation_data = validation_generator,
                   epochs=25
                   ,callbacks = [history, checkpoint, early])

Epoch 1/25
703/704 [============================>.] - ETA: 0s - loss: 3.1013 - acc: 0.2432
Epoch 00001: val_acc improved from -inf to 0.23994, saving model to Gabis_1549897763.h5
704/704 [==============================] - 222s 316ms/step - loss: 3.1012 - acc: 0.2433 - val_loss: 3.2554 - val_acc: 0.2399
Epoch 2/25
703/704 [============================>.] - ETA: 0s - loss: 3.0387 - acc: 0.2555
Epoch 00002: val_acc did not improve from 0.23994
704/704 [==============================] - 222s 315ms/step - loss: 3.0395 - acc: 0.2553 - val_loss: 3.3059 - val_acc: 0.2238
Epoch 3/25
703/704 [============================>.] - ETA: 0s - loss: 3.0157 - acc: 0.2688
Epoch 00003: val_acc improved from 0.23994 to 0.26308, saving model to Gabis_1549897763.h5
704/704 [==============================] - 223s 317ms/step - loss: 3.0146 - acc: 0.2690 - val_loss: 3.1733 - val_acc: 0.2631
Epoch 4/25
703/704 [============================>.] - ETA: 0s - loss: 2.9689 - acc: 0.2745
Epoch 00004: val_acc did not imp